In [ ]:

import numpy as np
import matplotlib.pyplot as plot
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5), (0.5))])

trainset = torchvision.datasets.MNIST('trainset', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST('testset', train = False,download=True,transform=transform)
train_loader = torch.utils.data.DataLoader(trainset,batch_size = 100,shuffle = True,num_workers=0)
test_loader = torch.utils.data.DataLoader(testset,batch_size = 100,shuffle = False,num_workers=0)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.features = nn.Sequential(nn.Conv2d(in_channels=1,
                                                out_channels=5,
                                                kernel_size=3,
                                                stride=1,
                                                padding=1), 
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(in_channels=5,
                                                out_channels=10,
                                                kernel_size=3,
                                                stride=1,
                                                padding=1), 
                                      nn.MaxPool2d(2, 2),
                                      nn.ReLU(inplace=True),
                                      nn.BatchNorm2d(10),
                                      nn.Conv2d(in_channels=10,
                                                out_channels=20,
                                                kernel_size=3,
                                                stride=1,
                                                padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.BatchNorm2d(20),
                                      nn.Conv2d(in_channels=20,
                                                out_channels=40,
                                                kernel_size=3,
                                                stride=1,
                                                padding=1),
                                      nn.MaxPool2d(2, 2),
                                      nn.ReLU(inplace=True),
                                      nn.BatchNorm2d(40))
        
        self.classifier = nn.Sequential(
            nn.Linear(7 * 7 * 40, 200),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(200, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 10))
        
    def forward(self, x):
        x = self.features(x)       
        x = x.view(-1, 7 * 7 * 40)
        x = self.classifier(x)
        return x

In [ ]:
model = Net()
losscalc=nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr = 3e-4,
                       weight_decay= 0.001)

In [ ]:
count = 0
loss_list = []
iteration_list = []
accuracy_list = []
for epoch in range(10):
    for i, data in enumerate(train_loader, 0):
        train, labels = data       
        optimizer.zero_grad()      
        outputs = model(train)      
        loss = losscalc(outputs, labels)       
        loss.backward()       
        optimizer.step()
        count += 1
        
        if count % 600 == 0:        
            correct = 0
            total = 0
            for i, data in enumerate(test_loader, 0):
                test, labels = data
                outputs = model(test)
                predicted = torch.max(outputs.data, 1)[1] 
                total += len(labels)               
                correct += (predicted == labels).sum()           
            accuracy = 100 * correct / float(total)
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
            print('Epoch: {}  Loss: {}  Accuracy: {} %'.format(count/600, loss.data, accuracy))

Epoch: 1.0  Loss: 0.027901282534003258  Accuracy: 97.70999908447266 %
Epoch: 2.0  Loss: 0.0706169605255127  Accuracy: 98.33999633789062 %
Epoch: 3.0  Loss: 0.010883302427828312  Accuracy: 98.51000213623047 %
Epoch: 4.0  Loss: 0.17128999531269073  Accuracy: 98.55000305175781 %
Epoch: 5.0  Loss: 0.008609485812485218  Accuracy: 98.69000244140625 %
Epoch: 6.0  Loss: 0.022639267146587372  Accuracy: 98.72000122070312 %
Epoch: 7.0  Loss: 0.06403082609176636  Accuracy: 98.62000274658203 %
Epoch: 8.0  Loss: 0.011927729472517967  Accuracy: 98.70999908447266 %
Epoch: 9.0  Loss: 0.008247327990829945  Accuracy: 98.7300033569336 %
Epoch: 10.0  Loss: 0.01265181228518486  Accuracy: 98.80999755859375 %
